
# 04 Inundation Mapping (Simplified)

This notebook demonstrates a simplified inundation mapping process. In a full flood forecasting system, predicted water levels from the hydrologic model would be overlaid on high-resolution Digital Elevation Models (DEMs) to delineate flooded areas.  

Due to data access constraints, we approximate the inundation model by creating a bounding box around the Greater Toronto Area (GTA) and flagging it as flooded on dates when the hydrologic model predicts a heavy precipitation event. This demonstration highlights the linkage between the forecasting component and spatial visualization of flood risk.


In [ ]:

from shapely.geometry import Polygon
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt

# Bounding box coordinates for Toronto area
bbox_coords = [
    (-79.7, 43.5),
    (-79.1, 43.5),
    (-79.1, 43.9),
    (-79.7, 43.9)
]
bbox_polygon = Polygon(bbox_coords)

# Load analytical data and model
analytical_df = pd.read_csv('data/processed/analytical_data.csv')
model = joblib.load('models/hydrologic_model.pkl')

# Prepare features for prediction
exclude_cols = ['Date', 'Next_Day_Precip', 'Flood_Event_Imminent']
feature_cols = [c for c in analytical_df.columns if c not in exclude_cols]
X_all = analytical_df[feature_cols].select_dtypes(include=['number']).fillna(0)

# Predict probabilities and labels
pred_proba = model.predict_proba(X_all)[:, 1]
pred_labels = (pred_proba > 0.5).astype(int)

# Combine with dates
results_df = pd.DataFrame({
    'Date': pd.to_datetime(analytical_df['Date'], errors='coerce'),
    'TrueLabel': analytical_df['Flood_Event_Imminent'],
    'PredProb': pred_proba,
    'PredLabel': pred_labels
})

# Find predicted flood dates
dates_with_flood = results_df.loc[results_df['PredLabel'] == 1, 'Date'].dt.date.unique()

print('Predicted flood dates:', dates_with_flood)

# Select a date for demonstration
if len(dates_with_flood) > 0:
    demo_date = sorted(dates_with_flood)[-1]
else:
    demo_date = None


In [ ]:

# Plot the Toronto bounding box and highlight flood status
plt.figure(figsize=(6, 6))

x_coords = [pt[0] for pt in bbox_coords] + [bbox_coords[0][0]]
y_coords = [pt[1] for pt in bbox_coords] + [bbox_coords[0][1]]

if demo_date:
    plt.fill(x_coords, y_coords, color='steelblue', alpha=0.5, edgecolor='black')
    plt.title(f'Predicted Flood Extent on {demo_date}')
else:
    plt.fill(x_coords, y_coords, color='lightgrey', edgecolor='black')
    plt.title('No Predicted Flood Events')

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.xlim(min(x_coords) - 0.05, max(x_coords) + 0.05)
plt.ylim(min(y_coords) - 0.05, max(y_coords) + 0.05)
plt.grid(True)
plt.show()
